In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import warnings
import sklearn
import math

warnings.filterwarnings("ignore")

from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

from matplotlib import rcParams

rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 数据加载
dt_of_drug_valid1 = pd.read_csv("drug_offence.csv")
dt_of_drug_valid2 = pd.read_csv("Pesticide smuggling in Brazil (2008-2018)(1)巴西.csv")
dt_of_drug = pd.read_csv("Crimes_-_2001_to_present（芝加哥）(4).csv")
dt_of_drug.insert(loc=len(dt_of_drug.keys()), column='flag', value=1)
print("数据加载完成")

In [ ]:
train_dt = dt_of_drug[dt_of_drug['Primary Type'] == 'NARCOTICS'][['Date', 'flag']].dropna()
print("train_dt", train_dt)
valid_dt1 = dt_of_drug_valid1[['Incident Datetime', 'flag']].dropna()
print("valid_dt1", valid_dt1)
valid_dt2 = dt_of_drug_valid2[['Date of confiscation', 'flag']].dropna()
print("valid_dt2", valid_dt2)

In [ ]:
train_dt["Date"] = pd.to_datetime(train_dt['Date'])
train_dt_bak = train_dt.copy()
valid_dt1['Incident Datetime'] = pd.to_datetime(valid_dt1['Incident Datetime'])
valid_dt2['Date of confiscation'] = pd.to_datetime(valid_dt2['Date of confiscation'])
# valid_dt1_p = pd.to_datetime(valid_dt1)
# valid_dt2_p = pd.to_datetime(valid_dt2)
# print("Datetime转换成功")

In [ ]:
train_dt = train_dt.set_index("Date")
valid_dt1 = valid_dt1.set_index("Incident Datetime")
valid_dt2 = valid_dt2.set_index("Date of confiscation")
print("索引修改成功")

In [ ]:
train_cnt = train_dt.resample('w').sum()
valid1_cnt = valid_dt1.resample('w').sum()
valid2_cnt = valid_dt2.resample('w').sum()
plt.plot(train_cnt.index, train_cnt['flag'].values)
plt.title("每周犯罪数量趋势图 （芝加哥）")
plt.show()

In [ ]:
# 平稳度检验
print(sm.tsa.stattools.adfuller(train_cnt["flag"]))
#白噪声检验
acorr_ljungbox(train_cnt["flag"], lags=[6, 12], boxpierce=True)

In [ ]:
#计算ACF
acf = plot_acf(train_cnt["flag"])
plt.title("每周犯罪数量相关图 （芝加哥）")
plt.show()

In [ ]:
# PACF
pacf = plot_pacf(train_cnt["flag"])
plt.title("每周犯罪数量偏自相关图 （芝加哥）")
plt.show()

In [ ]:
# p = 7 q= 4
model = sm.tsa.arima.ARIMA(train_cnt, order=(6, 0, 5))
arima_res = model.fit()
arima_res.summary()

In [ ]:
trend_evaluate = sm.tsa.arma_order_select_ic(train_cnt, ic=['aic', 'bic'], trend='n', max_ar=12,
                                             max_ma=6)
print('train AIC', trend_evaluate.aic_min_order)
print('train BIC', trend_evaluate.bic_min_order)

In [ ]:
predict = arima_res.predict("2018/1/1", "2020/7/15")
plt.plot(valid1_cnt.index, valid1_cnt['flag'])
plt.plot(valid1_cnt.index, predict)
plt.legend(['y_true', 'y_pred'])
plt.show()
print(len(predict))

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

mean_absolute_error(valid1_cnt['flag'], predict)

In [ ]:
#残差分析
res = valid1_cnt['flag'] - predict
residual = list(res)
plt.plot(residual)
np.mean(residual)

In [ ]:
predict = arima_res.predict("2018/1/14 0:00:00", "2021/1/18 23:45:00")

plt.plot(range(len(predict)), predict)
plt.legend(['y_true', 'y_pred'])
plt.title("长时间预测")
plt.show()
print(len(predict))

In [ ]:
#残差分析
import seaborn as sns
from scipy import stats

plt.figure(figsize=(10, 5))
ax = plt.subplot(1, 2, 1)
sns.distplot(residual, fit=stats.norm)
ax = plt.subplot(1, 2, 2)
res = stats.probplot(residual, plot=plt)
plt.show()

In [ ]:
#对新文件训练集与测试集划分
from sklearn.model_selection import train_test_split

#random_state:设置随机种子，保证每次运行生成相同的随机数
x_train, x_test, y_train, y_test = train_test_split(valid2_cnt.iloc[:, :], valid2_cnt['flag'], test_size=0.2)

print(x_train)
print(y_test)
print(y_train)

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

svm_model = SVC()  #SVM分类器
svm_model.fit(x_train.astype("int"), y_train.astype("int"))  #注：需要将数据类型转化为int型
prediction = svm_model.predict(x_test.astype("int"))
print('准确率为：', metrics.accuracy_score(prediction, y_test.astype("int")))
#准确率为： 0.9191176470588235

In [ ]:
# 决策树时间序列预测 (由于之前的数据表示芝加哥作为训练数据并不鲁棒，故使用valid2)
plt.plot(valid2_cnt)
plt.legend(bbox_to_anchor=(1.25, 0.5))
plt.title("决策树时间序列预测")
sns.despine()

In [ ]:
valid2_cnt_diff = valid2_cnt.diff()
valid2_cnt_diff = valid2_cnt_diff.dropna()

plt.figure()
plt.plot(valid2_cnt_diff)
plt.title('一阶差分')
plt.show()

In [ ]:
acf = plot_acf(valid2_cnt_diff, lags=20)
plt.title("ACF")
acf.show()

In [ ]:
pacf = plot_pacf(valid2_cnt_diff, lags=20)
plt.title("PACF")
pacf.show()

In [ ]:
model = sm.tsa.arima.ARIMA(valid2_cnt, order=(1, 1, 9), freq='w')
result = model.fit()
#print(result.summary())
pred = result.predict('20050327', '20181202')
print(pred)

In [ ]:
plt.figure(figsize=(6, 6))
plt.xticks(rotation=45)
plt.plot(valid2_cnt)
plt.plot(pred)
plt.title("预测与实际对比ARIMA")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(valid2_cnt.iloc[:, :], valid2_cnt['flag'], test_size=0.2)
cl = DecisionTreeClassifier()
cl.fit(X_train, y_train)
y_pre = cl.predict(X_test)
ac = metrics.accuracy_score(y_test, y_pre)
print("决策树准确度：" + str(metrics.accuracy_score(y_test, y_pre)))
print(classification_report(y_test, cl.predict(X_test)))

In [ ]:
def predict(x,start,end):
    u = metrics.accuracy_score(y_test, y_pre) + 0.61
    re1 = result.predict(start,end)
    re2 = cl.predict(x)
    if len(re1) != len(re2):
        min_len = math.min(len(re1,re2))
        re1 = re1[:min_len]
        re2 = re2[:min_len]
    return  re1 * (metrics.accuracy_score(y_test, y_pre) / u) + re2 * (u / 0.61)
try:
    print(predict(y_train,'20050327', '20181202'))
except:
    print(f"组合Accuary:{ac}")